In [69]:
# Imports

import os
import json
import gradio as gr
import base64
from PIL import Image
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
from dotenv import load_dotenv
from openai import OpenAI

In [3]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [4]:
system_prompt='You are an helpful assistant'
system_prompt+='You work at an airline called FLYYO'
system_prompt+='You always do everything in your best effort to help out customers'
system_prompt+='Incase they ask anything you dont know tell them you dont know'

In [6]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "kampala":"$450"}
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price for {destination_city} called")
    city = destination_city.lower()
    return ticket_prices.get(city, 'unknown')

In [7]:
get_ticket_price('kampala')

Tool get_ticket_price for kampala called


'$450'

In [22]:
# There's a particular dictionary structure that's required to describe our function:
# This makes it easy for openai to understand your tools/functions.

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
plane_times = {'morning':'Boeing 737', 'afternoon':'Airbus A320 family', 'Evening':'Boeing 777', 'night':'Global 8000'}

def avv_plane(a_time):
    print(f"Tool showing available plane in the {a_time} called")
    return plane_times.get(a_time.lower(), 'unknown')

In [10]:
print(avv_plane('morning'))

Tool showing available plane in the morning called
Boeing 737


In [11]:
plane_function = {
    'name':'avv_plane',
    'description':'Get the plane availabe at the time the customer wants to travel, Call this whenever you want to know the available plane in the time the user wants to travel.for example the customer asks "Which aeroplane/plane will be there in the morning"',
    'parameters':{
        'type':'object',
        'properties':{
            'a_time':{
                'type':'string',
                'description':'the time at which the customer wants to travel'
            }
        },
        'required':['a_time'],
        'additionalProperties':False
    }
}

In [12]:
tools = [{'type':'function', 'function':price_function} , {'type':'function', 'function':plane_function}]

In [24]:
def chat(message, history):
    messages = [{'role':'system', 'content':system_prompt}] + history + [{'role':'user', 'content':message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=='tool_calls':
        message = response.choices[0].message
        tool_response = handle_tool_call(message) # This handles what is returned by the tool call.
        messages.append(message)
        messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn = chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price for Kampala called


In [25]:
# A simple way to handle multiple tool calls using a for loop.

def handle_tool_call(message):
    for tool_call in message.tool_calls: # Looks for a given tool in message.tool_calls
        if tool_call.function.name == 'get_ticket_price': # In this case, this only runs if the tool called is get_ticket_price'
            arguments = json.loads(tool_call.function.arguments) # The name used for tool calls is the one used in the tool's function.
            city = arguments.get('destination_city')
            price = get_ticket_price(city)
            return {
                "role": "tool",       # Incase the tool call is successful, it will return this dict as the tool response.
                "content": f"The ticket price to {city} is ${price}", 
                "tool_call_id": tool_call.id,
                "city":city  # Passed in the city key and value so that we are able to capture it.
            }
            return None
        if tool_call.function.name == 'avv_plane':
            arguments = json.loads(tool_call.function.arguments)
            a_time = arguments.get('a_time')
            plane = avv_plane(a_time)
            return {
                "role": "tool", 
                "content": f"Available planes at {a_time}: {plane}",
                "tool_call_id": tool_call.id
            }
            return None

In [26]:
def artist(city):
    image_response=openai.images.generate(
        model='dall-e-3',
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
        size='1024x1024',
        n=1,
        response_format="b64_json"
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [27]:
def talker(message):
    custom_temp_dir = os.path.expanduser("~/Documents/temp_audio")
    os.environ['TEMP'] = custom_temp_dir
    
    if not os.path.exists(custom_temp_dir):
        os.makedirs(custom_temp_dir)
    
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    play(audio)

In [35]:
def chat(history):
    messages = [{"role": "system", "content": system_prompt}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None

    if response.choices[0].finish_reason=='tool_calls':
        message = response.choices[0].message
        tool_response = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)
        image = artist(tool_response['city'])
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    talker(reply)
    
    return history, image

In [73]:
def transcribe_audio(audio_file):
    """Convert audio to text using OpenAI Whisper"""
    if audio_file is None or not os.path.isfile(audio_file):
        print(f"[Error] Invalid file path: {audio_file}")
        return ""
    
    try:
        # Open the audio file and transcribe it
        with open(audio_file, "rb") as audio:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio,
                response_format="text"
            )
        return transcript.strip()
    except Exception as e:
        print(f"Transcription error: {e}")
        return ""

In [57]:
def process_audio_input(audio_file, history):
    """Process audio input: transcribe, then chat"""
    if audio_file is None:
        return "", history, None
        
    transcribed_text = transcribe_audio(audio_file)
    
    if not transcribed_text:
        return "", history, None
    
    # Add user message to history
    history = history or []
    history.append({"role": "user", "content": transcribed_text})
    
    # Process through existing chat system
    updated_history, image = chat(history)
    
    # Return empty audio (to clear the input), updated history, and image
    return None, updated_history, image

In [58]:
def do_entry(message, history):
    if not message.strip():
        return "", history
        
    history = history or []
    history.append({"role": "user", "content": message})
    
    return "", history

# NEW: Process text input through chat
def process_text_input(message, history):
    """Handle text input and process through chat"""
    if not message.strip():
        return "", history, None
    
    # Add to history
    history = history or []
    history.append({"role": "user", "content": message})
    
    # Process through chat
    updated_history, image = chat(history)
    
    return "", updated_history, image

In [74]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    
    with gr.Row():
        # Text input
        entry = gr.Textbox(label="Type your message:", placeholder="Type here or use voice input below...")
    
    with gr.Row():
        # Audio input
        audio_input = gr.Audio(
            sources=["microphone"],
            type="filepath",
            show_label=False,
            scale=3
            )
    
    with gr.Row():
        clear = gr.Button("Clear Chat")
    
    # Text input handling
    entry.submit(
        process_text_input,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot, image_output]
    )
    
    # Audio input handling
    audio_input.change(
        process_audio_input,
        inputs=[audio_input, chatbot],
        outputs=[audio_input, chatbot, image_output]
    )
    
    # Clear chat
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\OFFICIAL\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\OFFICIAL\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\OFFICIAL\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2157, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\OFFICIAL\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1947, in postprocess_data
    await processing_utils.async_move_files_to_cache(
  File "C:\Users\OFFICIAL\anaconda3\envs\llms\Lib\site-packages\gradio\processing_utils.py", line 648

In [77]:
!git --version

git version 2.47.1.windows.1


In [78]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../extras/community/prototype_signal.ipynb
	modified:   ../extras/trading/prototype_trader.ipynb
	modified:   ../week1/Guide to Jupyter.ipynb
	modified:   ../week1/Intermediate Python.ipynb
	modified:   ../week1/day1.ipynb
	modified:   ../week1/day2 EXERCISE.ipynb
	modified:   ../week1/day5.ipynb
	modified:   ../week1/troubleshooting.ipynb
	modified:   ../week1/week1 EXERCISE.ipynb
	modified:   day1.ipynb
	modified:   day2.ipynb
	modified:   day3.ipynb
	modified:   day4.ipynb
	modified:   day5.ipynb
	modified:   week2 EXERCISE.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../.env.txt
	../PesaPal_Documentation.txt
	bro_lingo.ipynb
	company_brochure.ipynb
	output_audio.mp3
	requirements.txt
	two_tools.ipynb

n

In [79]:
!git add two_tools.ipynb

In [80]:
!git commit -m "Heyy, its about the two tools simplifed method using a for loop"

[main 3e988bc] Heyy, its about the two tools simplifed method using a for loop
 1 file changed, 649 insertions(+)
 create mode 100644 week2/two_tools.ipynb


In [90]:
!git push

To https://github.com/aaron-official/llm_engineering.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/aaron-official/llm_engineering.git'
hint: Updates were rejected because the remote contains work that you do not
hint: have locally. This is usually caused by another repository pushing to
hint: the same ref. If you want to integrate the remote changes, use
hint: 'git pull' before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
